# Data Science Homework 1

For this homework we'll be looking at the data released by the State Bank of Pakistan. The data is the daily bank-wise and donor-wise receipts of the fund for the Daimer Bhasha and Mohmand Dam. You can find them in the following link: http://www.sbp.org.pk/notifications/FD/DamFund/Damfund.htm. Take a moment to look around the data and try to figure out what the possible challenges could be.

The main purpose of this homework is to teach how to scrape data from the web, clean it, and import it into Pandas for data analysis purposes. There are however some things to note:
1. As you can tell, the data is in PDF form. PDF is the most difficult to handle data format and if you get extremely broken CSV files, there isn't a need to worry, that's where the cleaning part comes in.
2. We'll be using an API to convert the data from PDF to CSV, and then from CSV to Pandas. There are, however, other ways to do this. The reason we wanted to do this method is two-fold
    * It will teach you how to communicate with APIs using Python, which will be a useful skill when you want to deploy your data models as an API so that it can work with other APIs that need those data models. Moreover, a lot of data you get in the real world is from APIs. 
    * The CSV will be extremely inconsistent, so it will give you immense practice with using regular-expressions, which are extremely important in the Data Science tool-kit.
    
Submit the notebooks in a similar format to the Labs: print the relevant output in each cell **only if it has an output. The initial scraping and converting does not have any output**, and name the notebooks as:
**rollnumber_HW1.ipynb** for e.g **20100237_H1.ipynb**

Please make sure you complete full parts (denoted by a Header each in this notebook) as the grading will be based on parts. Needless to say, do not copy someone else's code. In most Data Science careers, the main skill is not how good you are at coding, but how well you are able to use the tools at your disposal and what inferences you are able to make with the information that you have. Thus, while you might be able to do the HW by looking at someone else's code, unless you go through the actual thought process, you won't learn a lot.

We'll be using a lot of libraries in this tutorial, make sure you go through them so you understand what they are used for.

**NOTE: If you are more comfortable doing so, as I am, you can do the assignment on your preferred text editor on simple Python and then write the code neatly in a notebook.** Personally, I find Sublime/Vim easier to use than Jupyter, mostly since a lot of shortcuts there make coding much easier, while here the shortcuts are more about navigation and controlling your cells.

**The homework is to be done in pairs of 2.** 

**Naming convention: rollnumber1_rollnumber2_HW1.ipynb**

Total Marks: 100

## Part 0: Getting the Data

You can have a look at the data through the link given above. Download a few PDF files and go through the data to see what it looks like. How many columns are there, each, in the PDF files? Are there any inconsistencies? Any particular values that pop out that would need to be taken care of later in your cleaning? Think of all these questions when going through the initial PDF because they will prove really helpful when you can not figure out why there are so many "NaN" values in your final DataFrame.

## Part 1: Data Scraping              
Marks: 20

We'll be using what is called the *requests* model to get an HTML page, and then use *BeautifulSoup* to parse that HTML page such that we are able to to derive the appropriate information from it. I recommend you go through the documentation of each to learn more about how to use the libraries. 

* [Requests Documentation](http://docs.python-requests.org/en/master/)
* [BeautifulSoup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [BeautifulSoup + Requests Tutorial](https://www.pythonforbeginners.com/python-on-the-web/web-scraping-with-beautifulsoup)
* [BeautifulSoup](https://medium.freecodecamp.org/how-to-scrape-websites-with-python-and-beautifulsoup-5946935d93fe) Note that this tutorial is more detailed. I would highly recommend you go through this as well even though the library used here is urllib2 instead of requests (which you can do as well!). It also links to more web-scraping libraries like Scrapy for more complicated scraping.

In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import os
import re

# os is being imported so you can make a new directory. 

In [2]:
## Write code here that will:
    # Open each PDF link
    # Save the PDF in a directory in the same folder 
    
    
# Your code goes here #

url = "www.sbp.org.pk/notifications/FD/DamFund/Damfund.htm"
base_pdf_url = "http://" + (url.replace('/Damfund.htm', ''))
cwd = os.getcwd()

r  = requests.get("http://" +url)

data = r.text

soup = BeautifulSoup(data, "html.parser")

pdf_links = []

for link in soup.find_all('a'):
	if 'pdf' in link.get('href'):
		pdf_links.append(base_pdf_url + '/' + link.get('href'))

num_of_links = len(pdf_links)

if not os.path.exists('all_pdfs'):
	    os.makedirs('all_pdfs')

for i in xrange(0, num_of_links):
	r = requests.get(pdf_links[i], stream=True)
	match = re.search(r'\d{2}-\d{2}-\d{4}', pdf_links[i])
	filename = match.group()
	with open(cwd + '/all_pdfs/' + filename + '.pdf', 'wb') as f:
		f.write(r.content)

## Part 2: Converting from PDF to CSV
Marks: 15

You have two possible options between deciding what API to use for the conversion task.

The first option is communicating with an API called [Zamzar](https://www.zamzar.com) to send each PDF, ask them to convert it into CSV, and then download the converted CSV. They provide sample code to do everything from generating a simple request to starting a conversion job, checking for completion, and then downloading the finished file. You can find this information on the [Zamzar Documentation](https://developers.zamzar.com/docs) page.

**Important Information: **

The API only provides 100 points of free conversion, and each PDF to CSV conversion costs 3 points, that means with one account you can only convert **33** PDFs. However, this also means you have very little room to play around with this API, unless you have an extra email-address, so you need to be very careful when coding to communicate with this API. 

Moreover, the API only keeps the converted files for one day with a free account, so make sure you do this part in one go.

**Note: Using the Zamzar API grants a bonus of 10 marks. This will help if you are not able to complete this assignment, or it can be used up in a later assignment if you get 110/100 marks in this one.**

Another extremely simple API is the [PDF Tables](https://pdftables.com) API which is much simpler to use than the Zamzar API, however does not allow you to check the job for completion or for any intermediate steps. Moreover, this requires the installation of a library. Once again, they allow only 50 versions for free, but that is enough conversions for us. This [blog post](https://pdftables.com/blog/pdf-to-excel-with-python) will help you figure out how to convert the PDF to CSV using Python.

The cons of this API is that it will not really teach you any proper API communcation through requests since you do not have to navigate through any requests.

In [3]:
from requests.auth import HTTPBasicAuth
# import config
import glob

In [4]:
pdfs_folder = './all_pdfs/*.pdf'
api_key = []
api_key.append('bfbfea064c0e6241ee3714e30f0c0aa6ebadf9de')
api_key.append('c6dbc24c1a45d90f7a3bde7178401ffcceef8254')
endpoint = "https://sandbox.zamzar.com/v1/jobs"
if not os.path.exists('all_csvs'):
	os.makedirs('all_csvs')

# You need a list to store all the job_ids from the response of posting the conversion job, 
# if you are using the Zamzar API
job_ids = []
key = 0

# This piece of code shows you what glob does
for file_name in glob.glob(pdfs_folder):
    source_file = file_name
    target_format = "csv"
    file_content = {'source_file': open(source_file, 'rb')}
    data_content = {'target_format': target_format}
    response = requests.post(endpoint, data=data_content, files=file_content, auth=HTTPBasicAuth(api_key[key], ''))
    result = response.json()
    if 'errors' in result:
    	errors = result['errors']
    	errors_list = errors[0]
    	if errors_list['code'] == 25:
    		continue
    if 'errors' in result:
    	errors = result['errors']
    	errors_list = errors[0]
    	if errors_list['code'] == 24:
    		key += 1
    		response = requests.post(endpoint, data=data_content, files=file_content, auth=HTTPBasicAuth(api_key[key], ''))
    		result = response.json()

    job_ids.append(result['id'])

key = 0
for id in job_ids:
	endpoint = "https://sandbox.zamzar.com/v1/jobs/{}".format(id)
	response = requests.get(endpoint, auth=HTTPBasicAuth(api_key[key], ''))
	result = response.json()
	if 'errors' in result:
		e = result['errors']
		errors_list = e[0]
		if errors_list['code'] == 21:
			key += 1
    		response = requests.get(endpoint, auth=HTTPBasicAuth(api_key[key], ''))
    		result = response.json()
	status = result['status']
	if status == 'failed':
		continue
	while(status == 'converting'):
		response = requests.get(endpoint, auth=HTTPBasicAuth(api_key[key], ''))
		result = response.json()
		status = result['status']
	temp = result['target_files']
	file_id = temp[0]['id']
	file_name = temp[0]['name']

	local_filename = './all_csvs/' + file_name
	endpoint = "https://sandbox.zamzar.com/v1/files/{}/content".format(file_id)
	response = requests.get(endpoint, stream=True, auth=HTTPBasicAuth(api_key[key], ''))

	try:
	  with open(local_filename, 'wb') as f:
	    for chunk in response.iter_content(chunk_size=1024):
	      if chunk:
	        f.write(chunk)
	        f.flush()


	except IOError:
	  print "Error downloading CSV"

KeyError: 'id'

Below this cell write the code to download the completed files. First check if a job_id's status is completed and wait until it is. After it has been completed, download the file and save it.

The exact code required here is all in the documentation, the only additional task you have to do on your own is figure out a way to find out which file has just been received from the job_id, and name the local file.

**Please look at the Example JSON response in the [documentation](https://developers.zamzar.com/docs) to learn how to figure out the filenames, job status etc**

In [ ]:
## Your code goes here ##

## Part 3: Parsing the CSV File
Marks: 35

This is perhaps the most difficult part of the assignment, you have to follow a similar strategy to what you did the Udacity Lab 1. You can not simply use Pandas read_csv since the conversion is not perfect and there will be rows with different number of columns, which Pandas does not take care of.

### **Main Task:**
* Write a function that parses a CSV into a Pandas DataFrame
* Each DataFrame should consist of three columns with headers Bank, Donor_Name, and Amount
* The date should be retrieved from the given filename 
* The Donor_Name can be NaN, as it is in a lot of cases. But try to retrieve as much information as possible
* Remove all "Page of" rows
* Don't include the header rows (e.g. "SUPREME COURT FUND....") into the DataFrame
* The Amount should be converted into a Pandas numeric at the end

### Other info:
Some important resources for this part are (you can choose any one tutorial that you feel is easy to understand, they all cover roughly the same content):
* [RegEx Tutorial 1](https://www.regular-expressions.info/)
* [RegEx Tutorial 2](https://regexone.com/lesson/introduction_abcs)
* [RegEx Tutorial 3](https://www.rexegg.com/)
* [RegEx Cheatsheat](https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285)
* This [RegEx Editor](https://regex101.com/) is your best friend since you can test your expression separately on this

You will probably have to use the CSV reader in order to get all the rows of the file. You can learn more about it using this [tutorial](https://www.alexkras.com/how-to-read-csv-file-in-python/).

Some tips:
* First find out how many columns are in each row
* Print out rows which are longer than they should be (they should all be of length 3)
* Try to find patterns in how the data is spread, and what common problems exist in all rows
* Write some regex to try an extract the amount from the problem row and then:
    * Put the amount as the third column
    * Merge the rest of the string as a name of the donor in the 2nd column
* Also check if the rows with 3 columns are correctly formatted or not, many of them would probably not be.

In [ ]:
import csv
import re # To use regular expressions
import pandas as pd

example_file = './all_csvs/17-08-2018.csv' # Assuming the file is in the folder all_csvs and is named appropriately
# This is one of the most problematic files which is why I have included this in the example

In [ ]:
def parser(filename):
    # Complete this function
    
    with open(filename) as f:
        reader = csv.reader(f)
        data = [row for row in reader]
        
        ## Just an example of one way to use the CSV module
        
        return data

In [ ]:
# Remember, remove headers and convert all amounts to Numeric; if it can't be converted it needs to be NaN
def read_csv(filename):
    headers = ['Bank', 'Donor_Name','Amount']
    
    raw_data = parser(filename)
    
    df = # Your code goes here
    
    return df
    
print read_csv(example_file)

## Part 4: Importing Full Dataset
Marks: 10 

The only additional task in this part is to:
* Run the parser on all the files
* For each file **add a 'Date' column, which should be inferred from the filename**
* Concatenate each DataFrame into one large DataFrame. *Hint: concat*

In [ ]:
files = glob.glob('./all_csvs/*.csv')

full_data = # Your code goes her


print full_data.head()
print full_data.shape
print full_data.tail()

## Part 5: Data Integrity Checks
Marks: 20

* How many NaN values are there in each column? Why are they there? 
* What are the maximum and minimum values, is there anything peculiar about the max values?
* Are there any rows which are not NaN but should still be a different DataFrame altogether?
* Should these problem rows be removed? Can they be useful in other ways?